In [1]:
BRANCH = 'r1.12.0'

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]


In [2]:
# If you're not using Colab, you might need to upgrade jupyter notebook to avoid the following error:
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

# Please restart the kernel after running this cell

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

[NeMo W 2022-09-28 18:55:53 optimizers:77] Could not import distributed_fused_adam optimizer from Apex
[NeMo W 2022-09-28 18:55:54 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-09-28 18:55:54 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


# Task Description
Automatic Speech Recognition (ASR) systems typically generate text with no punctuation and capitalization of the words. 
This tutorial explains how to implement a model in NeMo that will predict punctuation and capitalization for each word in a sentence to make ASR output more readable and to boost performance of the named entity recognition, machine translation or text-to-speech models. 
We'll show how to train a model for this task using a pre-trained BERT model. 
For every word in our training dataset we’re going to predict:

- punctuation mark that should follow the word and
- whether the word should be capitalized

# Dataset
This model can work with any dataset as long as it follows the format specified below. 
The training and evaluation data is divided into *2 files: text.txt and labels.txt*. 
Each line of the **text.txt** file contains text sequences, where words are separated with spaces: [WORD] [SPACE] [WORD] [SPACE] [WORD], for example:



```
when is the next flight to new york
the next flight is ...
...
```



The **labels.txt** file contains corresponding labels for each word in text.txt, the labels are separated with spaces. Each label in labels.txt file consists of 2 symbols:

- the first symbol of the label indicates what punctuation mark should follow the word (where O means no punctuation needed);
- the second symbol determines if a word needs to be capitalized or not (where U indicates that the word should be upper cased, and O - no capitalization needed.)

In this tutorial, we are considering only commas, periods, and question marks the rest punctuation marks were removed. To use more punctuation marks, update the dataset to include desired labels, no changes to the model needed. 

Each line of the **labels.txt** should follow the format: 
[LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (for labels.txt). 
For example, labels for the above text.txt file should be:



```
OU OO OO OO OO OO OU ?U
OU OO OO OO ...
...
```



The complete list of all possible labels for this task used in this tutorial is: `OO, ,O, .O, ?O, OU, ,U, .U, ?U.`

## Download and preprocess the data¶

In this notebook we are going to use a subset of English examples from the [Tatoeba collection of sentences](https://tatoeba.org/eng) this script will download and preprocess the Tatoeba data [NeMo/examples/nlp/token_classification/get_tatoeba_data.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/data/get_tatoeba_data.py). Note, for further experiments with the model, set NUM_SAMPLES=-1 and consider including other datasets to improve model performance. 


In [4]:
DATA_DIR = 'data'
WORK_DIR = 'work'
MODEL_CONFIG = "punctuation_capitalization_config.yaml"

# model parameters
TOKENS_IN_BATCH = 1024
MAX_SEQ_LENGTH = 64
LEARNING_RATE = 0.00002
NUM_SAMPLES = 10000

In [5]:
## download get_tatoeba_data.py script to download and preprocess the Tatoeba data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_tatoeba_data.py'):
    print('Downloading get_tatoeba_data.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/data/get_tatoeba_data.py', WORK_DIR)
else:
    print ('get_tatoeba_data.py is already exists')

In [6]:
# download and preprocess the data
# --clean_dir flag deletes raw Tataoeba data, remove the flag to avoid multiple data downloads if you want to experiment with the dataset size
! python $WORK_DIR/get_tatoeba_data.py --data_dir $DATA_DIR --num_sample $NUM_SAMPLES --clean_dir

[NeMo I 2022-09-28 18:56:09 get_tatoeba_data:214] Downloading tatoeba dataset
[NeMo I 2022-09-28 18:56:09 get_tatoeba_data:38] Downloading https://downloads.tatoeba.org/exports/sentences.csv to data/sentences.csv
--2022-09-28 18:56:09--  https://downloads.tatoeba.org/exports/sentences.csv
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588527574 (561M) [application/octet-stream]
Saving to: ‘data/sentences.csv’

data/sentences.csv  100%[===================>] 561.26M  14.0MB/s    in 50s     

2022-09-28 18:56:59 (11.3 MB/s) - ‘data/sentences.csv’ saved [588527574/588527574]

[NeMo I 2022-09-28 18:56:59 get_tatoeba_data:218] Processing English sentences...
[NeMo I 2022-09-28 18:56:59 get_tatoeba_data:227] Splitting the tatoeba dataset into train and dev sets and creating labels and text files
[NeMo I 2022-09-28 18:56:5

after execution of the above cell, your data folder will contain the following 4 files needed for training (raw Tatoeba data could be present if `--clean_dir` was not used):
- labels_dev.txt
- labels_train.txt
- text_dev.txt
- text_train.txt


In [7]:
! ls -l $DATA_DIR

total 648
-rw-r--r-- 1 root root  46941 Sep 28 18:56 labels_dev.txt
-rw-r--r-- 1 root root 193605 Sep 28 18:56 labels_train.txt
-rw-r--r-- 1 root root  80515 Sep 28 18:56 text_dev.txt
-rw-r--r-- 1 root root 332061 Sep 28 18:56 text_train.txt


In [8]:
# let's take a look at the data 
print('Text:')
! head -n 5 $DATA_DIR/text_train.txt

print('\nLabels:')
! head -n 5 $DATA_DIR/labels_train.txt

Text:
let's try something
i have to go to sleep
i will be back soon
i'm at a loss for words
this is never going to end

Labels:
OU OO .O
OU OO OO OO OO .O
OU OO OO OO .O
OU OO OO OO OO .O
OU OO OO OO OO .O


As you see, `get_tatoeba_data.py` script provides not only downloads Tatoeba but also creates labels. If you wish to preprocess your own data, use [examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py) script:

In [ ]:
cwd = os.getcwd()
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/prepare_data_for_punctuation_capitalization.py \
    --source_file $DATA_DIR/text_train.txt \
    --output_dir $DATA_DIR/my_train_preprocessed

In [ ]:
!ls $DATA_DIR/my_train_preprocessed -l

# Tarred dataset

If your dataset is too large to be stored in memory, you can use tarred dataset. A tarred dataset is a collection of tarred files which contain batches ready for passing into a model.

All tar files will contain identical number of batches, so if number of batches in the dataset is not evenly divisible by parameter `--num_batches_per_tar_file` value, then up to `--num_batches_per_tar_file - 1` batches may be lost.

In [ ]:
# Number of lines in text and labels files
!wc -l $DATA_DIR/text_train.txt
!wc -l $DATA_DIR/labels_train.txt

In [ ]:
NEMO_ROOT = "~/NeMo"
!python $NEMO_ROOT/examples/nlp/token_classification/data/create_punctuation_capitalization_tarred_dataset.py \
    --text $DATA_DIR/text_train.txt \
    --labels $DATA_DIR/labels_train.txt \
    --output_dir $DATA_DIR/train_tarred \
    --num_batches_per_tarfile 5 \
    --tokens_in_batch 1024 \
    --lines_per_dataset_fragment 4000 \
    --tokenizer_name bert-base-uncased \
    --n_jobs 2

In [ ]:
!ls $DATA_DIR/train_tarred -l

In [ ]:
!ls $DATA_DIR/train_tarred/*.tar | wc -l  # number of tar files

In [ ]:
!ls $DATA_DIR/train_tarred/ | grep -v '.tar'  # all not tar files

If you wish to use tarred dataset, then you need to 
- pass metadata JSON file in config parameter `model.train_ds.tar_metadata_file`,
- set `model.train_ds.use_tarred_dataset=true`.

# Model Configuration

In the Punctuation and Capitalization Model, we are jointly training two token-level classifiers on top of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model: 
- one classifier to predict punctuation and
- the other one - capitalization.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, dataset and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

See [docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html#training-punctuation-and-capitalization-model) for full config description.

In [9]:
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [10]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

work/configs/punctuation_capitalization_config.yaml
pretrained_model: null
trainer:
  devices: 1
  num_nodes: 1
  max_epochs: 3
  max_steps: -1
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  precision: 16
  accelerator: gpu
  strategy: ddp
  enable_checkpointing: false
  logger: false
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
exp_manager:
  exp_dir: null
  name: Punctuation_and_Capitalization
  create_tensorboard_logger: true
  create_checkpoint_callback: true
model:
  class_labels:
    punct_labels_file: punct_label_ids.csv
    capit_labels_file: capit_label_ids.csv
  common_dataset_parameters:
    pad_label: O
    ignore_extra_tokens: false
    ignore_start_end: true
    punct_label_ids: null
    capit_label_ids: null
    label_vocab_dir: null
  train_ds:
    use_tarred_dataset: false
    ds_item: ???
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    tokens_in_batch: 15000
    max_s

# Setting up Data within the config

Among other things, the config file contains dictionaries called `common_dataset_parameters`, `train_ds` and `validation_ds`. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

Specify paths directories with train and dev datasets in parameters `train_ds.ds_item` and `validation_ds.ds_item`.

I you want to use multiple datasets for evaluation, specify paths to the directory(ies) with evaluation file(s) in the following way:

`model.validation_ds.ds_item=[PATH_TO_DEV1,PATH_TO_DEV2]` (Note no space between the paths and square brackets).

Also notice that some configs, including `model.train_ds.ds_item`, have `???` in place of values, this values are required to be specified by the user.

Let's now add the data directory path to the config.

In [11]:
# in this tutorial train and dev data is located in the same folder
config.model.train_ds.ds_item = DATA_DIR
config.model.validation_ds.ds_item=DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Let's first instantiate a Trainer object!

In [12]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Trainer config - 

devices: 1
num_nodes: 1
max_epochs: 3
max_steps: -1
accumulate_grad_batches: 1
gradient_clip_val: 0.0
precision: 16
accelerator: gpu
strategy: ddp
enable_checkpointing: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null



In [13]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

# Reduces maximum number of epochs to 1 for a quick training
config.trainer.max_epochs = 1

# Remove distributed training flags
config.trainer.strategy = None

trainer = pl.Trainer(**config.trainer)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


# Setting up a NeMo Experiment¶

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [14]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

[NeMo I 2022-09-28 18:57:57 exp_manager:291] Experiments will be logged at /NeMo/tutorials/nlp/nemo_experiments/Punctuation_and_Capitalization/2022-09-28_18-57-57
[NeMo I 2022-09-28 18:57:57 exp_manager:669] TensorboardLogger has been set up


[NeMo W 2022-09-28 18:57:57 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2274: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.")
    
[NeMo W 2022-09-28 18:57:57 exp_manager:919] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


'/NeMo/tutorials/nlp/nemo_experiments/Punctuation_and_Capitalization/2022-09-28_18-57-57'

# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model.

In [15]:
# complete list of supported BERT-like models
print(nemo_nlp.modules.get_pretrained_lm_models_list())

PRETRAINED_BERT_MODEL = "biomegatron-bert-345m-uncased"

['bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased', 'distilbert-base-uncased', 'distilbert-base-uncased-distilled-squad', 'distilbert-base-cased', 'distilbert-base-cased-distilled-squad', 'distilbert-base-german-cased', 'distilbert-base-multilingual-cased', 'distilbert-base

In [16]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.train_ds.num_samples = NUM_SAMPLES
config.model.validation_ds.num_samples = NUM_SAMPLES


Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [17]:
# initialize the model
# during this stage, the dataset and data loaders we'll be prepared for training and evaluation
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)

[NeMo W 2022-09-28 18:58:20 lm_utils:80] biomegatron-bert-345m-uncased is not in get_pretrained_lm_models_list(include_external=False), will be using AutoModel from HuggingFace.


[NeMo I 2022-09-28 18:58:20 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo.
[NeMo I 2022-09-28 18:58:20 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo
[NeMo I 2022-09-28 18:58:20 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-09-28 18:58:22 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-09-28 18:58:22 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-09-28 18:58:22 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-09-28 18:58:22 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-09-28 18:58:22 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-09-28 18:58:22 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-09-28 18:58:22 megatron_init:231

[NeMo W 2022-09-28 18:58:22 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-09-28 18:58:22 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-uncased, custom vocab file: /tmp/tmpxal6fgg_/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, and merges file: None
[NeMo I 2022-09-28 18:58:22 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-uncased, vocab_file: /tmp/tmpxal6fgg_/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


...

[NeMo I 2022-09-28 18:58:29 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo.
[NeMo I 2022-09-28 18:58:29 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo
[NeMo I 2022-09-28 18:58:29 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-09-28 18:58:31 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-09-28 18:58:31 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-09-28 18:58:31 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-09-28 18:58:31 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-09-28 18:58:31 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-09-28 18:58:31 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-09-28 18:58:31 megatron_init:231

[NeMo W 2022-09-28 18:58:31 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-09-28 18:58:31 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-uncased, custom vocab file: /tmp/tmpxc4zsave/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, and merges file: None
[NeMo I 2022-09-28 18:58:31 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-uncased, vocab_file: /tmp/tmpxc4zsave/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-09-28 18:58:31 megatron_base_model:185] Padded vocab_size: 30592, original vocab_size: 30522, dummy tokens: 70.
[NeMo I 2022-09-28 18:58:32 save_restore_connector:243] Model MegatronBertModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo.


## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [18]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

To use tensorboard, please use this notebook in a Google Colab environment.


In [19]:
# start the training
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4]


[NeMo I 2022-09-28 18:58:49 modelPT:597] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        eps: 1e-08
        foreach: None
        lr: 2e-05
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2022-09-28 18:58:49 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7f7010d526d0>" 
    will be used during training (effective maximum steps = 114) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 114
    )



  | Name             | Type              | Params
-------------------------------------------------------
0 | metrics          | ModuleDict        | 0     
1 | bert_model       | MegatronBertModel | 334 M 
2 | punct_classifier | TokenClassifier   | 4.1 K 
3 | capit_classifier | TokenClassifier   | 2.0 K 
4 | loss             | CrossEntropyLoss  | 0     
5 | agg_loss         | AggregatorLoss    | 0     
-------------------------------------------------------
334 M     Trainable params
0         Non-trainable params
334 M     Total params
668.340   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo I 2022-09-28 18:58:52 punctuation_capitalization_model:333] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         77.75      32.26      45.60        964
    , (label_id: 1)                                          1.01      15.38       1.89         26
    . (label_id: 2)                                         10.30      15.45      12.36        110
    ? (label_id: 3)                                          0.00       0.00       0.00         19
    -------------------
    micro avg                                               29.67      29.67      29.67       1119
    macro avg                                               22.27      15.78      14.96       1119
    weighted avg                                            68.02      29.67      40.54       1119
    
[NeMo I 2022-09-28 18:58:52 punctuation_capitalization_model:334] Capitalization rep

Training: 0it [00:00, ?it/s]

[NeMo W 2022-09-28 18:58:54 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:135: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
      warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
    


Validation: 0it [00:00, ?it/s]

[NeMo I 2022-09-28 18:59:05 punctuation_capitalization_model:333] Punctuation report: 
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         97.44      99.96      98.69      13311
    , (label_id: 1)                                         72.00       5.81      10.75        310
    . (label_id: 2)                                         92.36      96.17      94.23       1646
    ? (label_id: 3)                                         97.23      64.74      77.73        380
    -------------------
    micro avg                                               96.84      96.84      96.84      15647
    macro avg                                               89.76      66.67      70.35      15647
    weighted avg                                            96.40      96.84      95.97      15647
    
[NeMo I 2022-09-28 18:59:05 punctuation_capitalization_model:334] Capitalization rep

Batch building: 100%|██████████| 114/114 [00:00<00:00, 749.64batch/s]
Epoch 0, global step 114: 'val_loss' reached 0.13744 (best 0.13744), saving model to '/NeMo/tutorials/nlp/nemo_experiments/Punctuation_and_Capitalization/2022-09-28_18-57-57/checkpoints/Punctuation_and_Capitalization--val_loss=0.1374-epoch=0.ckpt' as top 3
`Trainer.fit` stopped: `max_epochs=1` reached.


In [21]:
model.save_to('trained.nemo')

In [30]:
path2file = 'new.nemo'

In [25]:
import tempfile
import tarfile

In [31]:

with tempfile.TemporaryDirectory() as tmpdir:
        tar_header = "r:"
        try:
            tar_test = tarfile.open(path2file, tar_header)
            tar_test.close()
        except tarfile.ReadError:
          
            tar_header = "r:gz"
        tar = tarfile.open(path2file, tar_header)
        tar.extractall(path=tmpdir)
        tar.close()
        nemo_nlp.models.PunctuationCapitalizationModel._set_model_restore_state(is_being_restored=True, folder=tmpdir)
        config = nemo_nlp.models.PunctuationCapitalizationModel.restore_from(path2file,trainer=trainer,return_config=True)
        from nemo.utils.app_state import AppState
        print(AppState().nemo_file_folder)
        x = nemo_nlp.models.PunctuationCapitalizationModel.from_config_dict(config,trainer=trainer)

[NeMo W 2022-09-28 19:11:48 lm_utils:80] biomegatron-bert-345m-uncased is not in get_pretrained_lm_models_list(include_external=False), will be using AutoModel from HuggingFace.


/tmp/tmpxrxdw2y8
[NeMo I 2022-09-28 19:11:48 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo.
[NeMo I 2022-09-28 19:11:48 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo
[NeMo I 2022-09-28 19:11:48 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-09-28 19:11:49 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-09-28 19:11:49 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-09-28 19:11:49 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-09-28 19:11:49 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-09-28 19:11:49 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-09-28 19:11:49 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-09-28 19:11:49 

[NeMo W 2022-09-28 19:11:49 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-09-28 19:11:49 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-uncased, custom vocab file: /tmp/tmpr351xi4t/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, and merges file: None
[NeMo I 2022-09-28 19:11:49 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-uncased, vocab_file: /tmp/tmpr351xi4t/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


...

[NeMo I 2022-09-28 19:11:51 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo.
[NeMo I 2022-09-28 19:11:51 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo
[NeMo I 2022-09-28 19:11:51 common:910] Instantiating model from pre-trained checkpoint
[NeMo I 2022-09-28 19:11:52 megatron_init:204] Rank 0 has data parallel group: [0]
[NeMo I 2022-09-28 19:11:52 megatron_init:207] All data parallel group ranks: [[0]]
[NeMo I 2022-09-28 19:11:52 megatron_init:208] Ranks 0 has data parallel rank: 0
[NeMo I 2022-09-28 19:11:52 megatron_init:216] Rank 0 has model parallel group: [0]
[NeMo I 2022-09-28 19:11:52 megatron_init:217] All model parallel group ranks: [[0]]
[NeMo I 2022-09-28 19:11:52 megatron_init:227] Rank 0 has tensor model parallel group: [0]
[NeMo I 2022-09-28 19:11:52 megatron_init:231

[NeMo W 2022-09-28 19:11:52 modelPT:217] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2022-09-28 19:11:52 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-bert-345m-uncased, custom vocab file: /tmp/tmpj4850u44/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, and merges file: None
[NeMo I 2022-09-28 19:11:52 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-large-uncased, vocab_file: /tmp/tmpj4850u44/e10447bf5e1c4338a20d04292c98fafb_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-09-28 19:11:53 megatron_base_model:185] Padded vocab_size: 30592, original vocab_size: 30522, dummy tokens: 70.
[NeMo I 2022-09-28 19:11:53 save_restore_connector:243] Model MegatronBertModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.12.0/BioMegatron345mUncased/8b5a30458454d34ea3ad6fb53835f214/BioMegatron345mUncased.nemo.


# Training using tarred dataset

In [ ]:
config = OmegaConf.load(config_path)
config.model.train_ds.ds_item = f'{DATA_DIR}/train_tarred'
config.model.train_ds.use_tarred_dataset = True
# Only metadata file name is required if `use_tarred_dataset=true`.
config.model.train_ds.tar_metadata_file = 'metadata.punctuation_capitalization.tokens1024.max_seq_length512.bert-base-uncased.json'
config.model.validation_ds.ds_item = DATA_DIR
del config.model.test_ds  # We do not have test data, only train and dev

# Trainer
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator
config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.max_epochs = 1
config.trainer.strategy = None

# Exp manager
config.exp_manager.explicit_log_dir = 'tarred_experiment'

config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.validation_ds.tokens_in_batch = TOKENS_IN_BATCH
config.model.optim.lr = LEARNING_RATE
config.model.validation_ds.num_samples = NUM_SAMPLES

In [ ]:
trainer = pl.Trainer(**config.trainer)
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.PunctuationCapitalizationModel(cfg=config.model, trainer=trainer)
trainer.fit(model)

# Inference using a pretrained model

To see how the model performs, let’s run inference on a few examples.

In [ ]:
print(f"Available_models: {nemo_nlp.models.PunctuationCapitalizationModel.get_available_model_names()}\n")

pretrained_model = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained("punctuation_en_distilbert")
# define the list of queiries for inference
queries = [
        'we bought four shirts and one mug from the nvidia gear store in santa clara',
        'what can i do for you today',
        'how are you',
        'how is the weather in',
    ]
inference_results = pretrained_model.add_punctuation_capitalization(queries)
print()

for query, result in zip(queries, inference_results):
    print(f'Query   : {query}')
    print(f'Combined: {result.strip()}\n')

Inference for large amounts of text can be performed by a script [examples/nlp/token_classification/punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py)

```
python punctuate_capitalize_infer.py \
    --input_manifest <PATH/TO/INPUT/MANIFEST> \
    --output_manifest <PATH/TO/OUTPUT/MANIFEST> \
    --pretrained_name punctuation_en_bert \
    --max_seq_length 64 \
    --margin 16 \
    --step 8
```

`<PATH/TO/INPUT/MANIFEST>` is a path to NeMo [ASR manifest](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/asr/datasets.html) which contains text in which you need to restore punctuation and capitalization. If manifest contains `'pred_text'` key, then `'pred_text'` elements will be processed. Otherwise, punctuation and capitalization will be restored in `'text'` elements.

`<PATH/TO/OUTPUT/MANIFEST>` is a path to NeMo ASR manifest into which result will be saved. The text with restored
punctuation and capitalization is saved into `'pred_text'` elements if `'pred_text'` key is present in
input manifest. Otherwise result will be saved into `'text'` elements.

Alternatively you can pass text for restoring punctuation and capitalization as plain text. See help for parameters `--input_text` and `--output_text` of the script
[punctuate_capitalize_infer.py](https://github.com/NVIDIA/NeMo/blob/stable/examples/nlp/token_classification/punctuate_capitalize_infer.py).

The script `examples/nlp/token_classification/punctuate_capitalize_infer.py` can restore punctuation and capitalization in a text of arbitrary length. Long sequences are split into segments
`--max_seq_length - 2` tokens each. Each segment starts and ends with `[CLS]` and `[SEP]`
tokens correspondingly. Every segment is offset to the previous one by `--step` tokens. For example, if
every character is a token, `--max_seq_length=5`, `--step=2`, then text `"hello"` will be split into
segments `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]`.

If segments overlap, then predicted probabilities for a token present in several segments are multiplied before selecting the best candidate.

Splitting leads to pour performance of a model near edges of segments. Use parameter `--margin` to discard `--margin`
probabilities predicted for `--margin` tokens near segment edges. For example, if every character is a token, `--max_seq_length=5`, `--step=1`, `--margin=1`, then text `"hello"` will be split into segments `[['[CLS]', 'h', 'e', 'l', '[SEP]'], ['[CLS]', 'e', 'l', 'l', '[SEP]'], ['[CLS]', 'l', 'l', 'o', '[SEP]']]`. Before calculating actual predictions, probabilities for tokens marked by asterisk are removed: `[['[CLS]', 'h', 'e', 'l'*, '[SEP]'*], ['[CLS]'*, 'e'*, 'l', 'l'*, '[SEP]'*], ['[CLS]'*, 'l'*, 'l', 'o', '[SEP]']]`.

You may use parameters `max_seq_length`, `step`, and `margin` in `add_punctuation_capitalization` method.

The text in the following example is from IWSLT 2019 test dataset.

In [ ]:
inference_results = pretrained_model.add_punctuation_capitalization(
    [
        "each of these songs represents a scene a movement in some cases a sonic revolution that completely altered the "
        "course of popular music they're all also calling cards almost for those cities songs totally linked with their "
        "city's identity and might be why you probably consider them to be music cities now the magical mythical thing "
        "the thing we kind of all love about stories like these is that those cities weren't doing anything in particular "
        "to make those moments happen there's no formula for capturing lightning in a bottle a formula didn't give us "
        "grunge music or introduce tupock to dr dray and there's definitely no blueprint for how to open your record "
        "business in a south memphis neighborhood that turns out is home to booker t jones william bell and albert king "
        "so this is just something that happens and right when the stars perfectly align great music just happens and "
        "in the meantime new york and nashville can churn out the hits that come through our radios define our "
        "generations and soundtrack our weddings and our funerals and everything in between i don't know about you but "
        "the very idea of that is just deadly boring to me there are musicians all around you making powerful important "
        "music and thanks to the internet and it's limitless possibilities for creators to create music and fans to "
        "discover that music those zyite guy songs don't have to be handed down to us from some conference room full of "
        "songwriters in a corporate high rise but also and more importantly we can't decide that it's just something that "
        "happens because music is about so much more than hits those big iconic moments that change everything it's more "
        "than just entertainment for so many of us music is truly a way to navigate life a means of self expression sure "
        "but it also helps us find ourrselfel worse and figure out who we are it connects us with other people as almost "
        "nothing else can across language barriers across social and cultural and economic divides music makes us smarter "
        "and healthier and happier music is necessary what if you lived in a city that believed that that said we're not "
        "waiting for that hit song to define us we're a music city because music is necessary by seeing music as "
        "necessary a city can build two things 1st an ecosystem to support the development of professional musicians "
        "and music business and 2nd a receptive and engaged audience to sustain them and those are the two critical "
        "elements of a music city a city whose leaders recognize the importance of music for our development as "
        "individuals our connection as a community and our viability as a vibrant place to live see smart cities music "
        "cities know that thriving night lifef a creative class culture is what attracts young talented people to cities "
        "it's what brings that lightning and no we can't predict the next egg that will hatch but we can create a city "
        "that acts like an incubator to do that 1st we got to know what we've got that means identifying and quantifying "
        "our assets we need to know them backward and forward from who and what and where they are to what their impact "
        "is on the economy let's count our recording studios and our record labels our historic landmarks and our "
        "hardcore punk clubs we should count monthly free jazz nights and weekly folk jams music schools artist "
        "development instrument shops every lay than every luther music museums open once a open year round and music "
        "festivals open just one weekend year now ideally through this process we'll create an actual asset map "
        "dropping a pin for each one allowing us to see exactly what we've got and where organic momentum is already "
        "happening because it's not enough to paint in broad strokes here when it comes to specific support for music "
        "locally and a broad understanding of a music brand nationally you've got to have the receipts next we'll need "
        "to identify our challenges now it's important to knowe that for the most part this won't be just the opposite "
        "of step one we won't gain a whole lot by simply thinking about what's missing from our map instead we need to "
        "approach this more holistically there are lots of music venues on our map awesome but are they struggling do "
        "we have a venue ladder which just means can an artist starting out at a coffee house open mike see a clear path "
        "for how they'll grow from that 25 seat room to a hundred seat room and so on or are we expecting them to go from "
        "a coffee house to a colossum maybe our challenges lie in city infrastructure public transportation affordable "
        "housing maybe like in london where the number of music venues went from 400 in 2010 to 100 in 2015 we need to "
        "think about protections against gentrification the mayor of london in december of last year actually added "
        "something called the agent of change principle to the city's comprehensive plan the name says it all if a real "
        "estate developer wants to build condos next to an existing music venue the developer is the agent of change they "
        "have to take the necessary steps for noise mitigation next and this is a very big one we need leadership and we "
        "need a strategy now we know there's a lot of magic in this mix a lot of right people right place right time and "
        "that will never stop being an important element of the way music is made the way some of the best most enduring "
        "music is made but there cannot be a leadership vacuum in 2018 surriving music cities don't often happen and "
        "don't have to happen accidentally we need elected officials who recognize the power of music and elevate the "
        "voices of creatives and they're ready to put a strategy in place in music cities from berlin to paris to bogata "
        "music advisory councils ensure that musicians have a seat at the table they're volunteer councils and they work "
        "directly with a designated advocate inside of city hall or even the chamber of commerce the strongest strategies "
        "will build music community supports like this one inward while also exporting music outward they go hand in hand "
        "when we look inward we create that place that musicians want to live and when we look outward we build "
        "opportunities for them to advance their career while also driving attention back to our city and leveraging music "
        "as a talent attraction tool and here's something else that will help with that we've got to figure out who we are "
        "now when i say austin you probably think live music capital and why because in 1991 leadership in austin saw "
        "something percolating with an existing asset and they chose to own it by recognizing that momentum naming it and "
        "claiming it they inevitably caused more live music venues to open existing spaces to add live music to their "
        "repertoire and they created a swell of civic buy in around the idea which meant that it wasn't just a slogan in "
        "some tourism pamphlet was something that locals really started to believe and take pride in now generally "
        "speaking what austin created is just an assets based narrative and when we think back to step one we know that "
        "every city will not tick every box many cities won't have recording studios like memphis or a songwriter and "
        "publishing scene like nashville and that's not a deal breaker we simply have to find the momentum happening in "
        "our city what are our unique assets in comparison to no other place so if all of that sounds like something "
        "you'd like to happen where you live here are three things you can do to move the needle 1st you can use your "
        "feet your ears and your dollars show up be that receptive and engaged audience that is so necessary for a music "
        "city to thrive pay a cover charge buy a record discover new music and please take your friends two you can use "
        "your voice buy in to the assets based narrative talk about and celebrate what your city has and three you can "
        "use your vote seek out leadership that doesn't just pay lip service to your city's music but recognizes its "
        "power and is prepared to put a strategy in place to elevate it grow it and build collaboration no there really "
        "is no telling what city could be defined by a certain scene or a certain song in the next decade but as much "
        "as we absolutely cannot predict that what we absolutely can predict is what happens when we treat music as "
        "necessary and we work to build a music city and that is a place where i want to live thank you"
    ],
    max_seq_length=128,
    step=8,
    margin=16,
    batch_size=32,
)
print(inference_results[0])

## Training Script

If you have NeMo installed locally, you can also train the model with [nlp/token_classification/punctuation_capitalization_train_evaluate.py](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/token_classification/punctuation_capitalization_train_evaluate.py).

To run training script, use:

`python punctuation_capitalization_train_evaluate.py model.train_ds.ds_item=PATH_TO_TRAIN_DATA_DIR`

Set NUM_SAMPLES=-1 and consider including other datasets to improve the performance of the model.

# Finetuning model with your data

When we were training the model from scratch, the datasets were prepared for training during the model initialization. When we are using a pretrained Punctuation and Capitalization model, before training, we need to setup training and evaluation data.

In [ ]:
# let's reload our pretrained model
pretrained_model = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained('punctuation_en_distilbert')

# setup train and validation Pytorch DataLoaders
pretrained_model.update_config_after_restoring_from_checkpoint(
    train_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_train.txt',
        'labels_file': 'labels_train.txt',
        'tokens_in_batch': 1024,
    },
    validation_ds={
        'ds_item': DATA_DIR,
        'text_file': 'text_dev.txt',
        'labels_file': 'labels_dev.txt',
        'tokens_in_batch': 1024,
    },
)

# and now we can create a PyTorch Lightning trainer and call `fit` again
# for this tutorial we are setting fast_dev_run to True, and the trainer will run 1 training batch and 1 validation batch
# for actual model training, disable the flag
fast_dev_run = True
trainer = pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
pretrained_model.set_trainer(trainer)
pretrained_model.setup_training_data()
pretrained_model.setup_validation_data()
trainer.fit(pretrained_model)